In [1]:
import numpy as np
import plotly.graph_objects as go
from IPython.display import Audio

In [2]:
class Shape:
    def __init__(self, refresh_rate=1000, carrier_rate=16000, amplification=[1., 1.]):
        self.refresh_rate = refresh_rate
        self.carrier_rate = carrier_rate
        self.amplification = amplification
    
    def play(self, duration):
        signal = self.signal(duration)
        return Audio(signal, rate=self.carrier_rate)
    
    def show(self):
        derivative = self.derivative(1/self.refresh_rate)
        return go.Figure(layout={
            'title': f'{type(self).__name__}',
            'autosize': False,
            'width': 700,
            'height': 700,
            'template': 'plotly_dark'
        }).add_trace(go.Scatter(
            x=derivative[0],
            y=derivative[1]
        ))
    
    def signal(self, duration):
        derivative = self.derivative(duration)
        integral = np.cumsum(derivative, axis=1)
        return integral # TODO: could subtract rolling mean or high-pass to stop this from wandering too far from 0
    
    def derivative(self, duration):
        sequence = np.linspace(
            start=0,
            stop=duration * self.refresh_rate,
            num=int(duration * self.carrier_rate)
        )
        signal = np.stack((self.dx(sequence), self.dy(sequence)), axis=-1)
        signal *= self.amplification
        signal = np.moveaxis(signal, -1, 0)
        return signal

In [3]:
class Circle(Shape):
    def x(self, sequence):
        return np.sin(2. * np.pi * sequence)
    
    def y(self, sequence):
        return np.cos(2. * np.pi * sequence)

In [4]:
circle = Circle(amplification=[1, 4])

In [5]:
circle.show()

In [6]:
circle.play(duration=1.0)